In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

In [3]:
!wget $data -O data-week-3.csv

--2023-09-30 22:10:08--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1,4M) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>]   1,41M  1022KB/s    in 1,4s    

2023-09-30 22:10:10 (1022 KB/s) - ‘data-week-3.csv’ saved [1475504/1475504]



In [4]:
df = pd.read_csv('data-week-3.csv')
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [5]:
df.describe()

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP
count,11914.000000,11845.00000,11884.000000,11908.000000,11914.000000,11914.000000,11914.000000,1.191400e+04
mean,2010.384338,249.38607,5.628829,3.436093,26.637485,19.733255,1554.911197,4.059474e+04
std,7.579740,109.19187,1.780559,0.881315,8.863001,8.987798,1441.855347,6.010910e+04
min,1990.000000,55.00000,0.000000,2.000000,12.000000,7.000000,2.000000,2.000000e+03
25%,2007.000000,170.00000,4.000000,2.000000,22.000000,16.000000,549.000000,2.100000e+04
50%,2015.000000,227.00000,6.000000,4.000000,26.000000,18.000000,1385.000000,2.999500e+04
75%,2016.000000,300.00000,6.000000,4.000000,30.000000,22.000000,2009.000000,4.223125e+04
max,2017.000000,1001.00000,16.000000,4.000000,354.000000,137.000000,5657.000000,2.065902e+06


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine Fuel Type   11911 non-null  object 
 4   Engine HP          11845 non-null  float64
 5   Engine Cylinders   11884 non-null  float64
 6   Transmission Type  11914 non-null  object 
 7   Driven_Wheels      11914 non-null  object 
 8   Number of Doors    11908 non-null  float64
 9   Market Category    8172 non-null   object 
 10  Vehicle Size       11914 non-null  object 
 11  Vehicle Style      11914 non-null  object 
 12  highway MPG        11914 non-null  int64  
 13  city mpg           11914 non-null  int64  
 14  Popularity         11914 non-null  int64  
 15  MSRP               11914 non-null  int64  
dtypes: float64(3), int64(5

In [7]:
columns=['Make','Model','Year','Engine HP','Engine Cylinders','Transmission Type','Vehicle Style','highway MPG','city mpg']

In [8]:
df_filt=df[columns]

In [9]:
df_filt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine HP          11845 non-null  float64
 4   Engine Cylinders   11884 non-null  float64
 5   Transmission Type  11914 non-null  object 
 6   Vehicle Style      11914 non-null  object 
 7   highway MPG        11914 non-null  int64  
 8   city mpg           11914 non-null  int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 837.8+ KB


In [10]:
df_filt.columns = df_filt.columns.str.replace(' ', '_').str.lower()
categorical_columns = list(df_filt.dtypes[df_filt.dtypes == 'object'].index)
for c in categorical_columns:
    df_filt[c] = df_filt[c].str.lower().str.replace(' ', '_')

/tmp/ipykernel_73759/1661914649.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt[c] = df_filt[c].str.lower().str.replace(' ', '_')


In [11]:
df_filt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               11914 non-null  object 
 1   model              11914 non-null  object 
 2   year               11914 non-null  int64  
 3   engine_hp          11845 non-null  float64
 4   engine_cylinders   11884 non-null  float64
 5   transmission_type  11914 non-null  object 
 6   vehicle_style      11914 non-null  object 
 7   highway_mpg        11914 non-null  int64  
 8   city_mpg           11914 non-null  int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 837.8+ KB


In [12]:
df_filt.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
dtype: int64

In [13]:
df_filt.head(3)

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg
0,bmw,1_series_m,2011,335.0,6.0,manual,coupe,26,19
1,bmw,1_series,2011,300.0,6.0,manual,convertible,28,19
2,bmw,1_series,2011,300.0,6.0,manual,coupe,28,20


In [14]:
df_filt[['price']]=df[['MSRP']]

/tmp/ipykernel_73759/1874659937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt[['price']]=df[['MSRP']]


In [15]:
base = ['engine_hp', 'engine_cylinders']

df_filt[base] = df_filt[base].fillna(0).values

/tmp/ipykernel_73759/376422996.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt[base] = df_filt[base].fillna(0).values


In [16]:
df_filt.transmission_type.value_counts()

transmission_type
automatic           8266
manual              2935
automated_manual     626
direct_drive          68
unknown               19
Name: count, dtype: int64

In [17]:
df_filt.transmission_type.mode()

0    automatic
Name: transmission_type, dtype: object

In [18]:
numerical=['engine_hp','year', 'engine_cylinders', 'highway_mpg', 'city_mpg']
df_filt[numerical].corr()

,engine_hp,year,engine_cylinders,highway_mpg,city_mpg
engine_hp,1.000000,0.338714,0.774851,-0.415707,-0.424918
year,0.338714,1.000000,-0.040708,0.258240,0.198171
engine_cylinders,0.774851,-0.040708,1.000000,-0.614541,-0.587306
highway_mpg,-0.415707,0.258240,-0.614541,1.000000,0.886829
city_mpg,-0.424918,0.198171,-0.587306,0.886829,1.000000


In [19]:
df_filt['above_average']=(df_filt.price > df_filt.price.mean())

/tmp/ipykernel_73759/3237790816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt['above_average']=(df_filt.price > df_filt.price.mean())


In [20]:
df_filt['above_average']=df_filt['above_average'].astype(int)

/tmp/ipykernel_73759/1767756741.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt['above_average']=df_filt['above_average'].astype(int)


In [21]:
df_full_train, df_test = train_test_split(df_filt, test_size=0.2, random_state=42) 
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [22]:
def mutual_info_target_score(series):
    return round(mutual_info_score(series, df_full_train.above_average),2)

In [23]:
mi = df_full_train[categorical_columns].apply(mutual_info_target_score)
mi.sort_values(ascending=False)

model                0.46
make                 0.24
vehicle_style        0.08
transmission_type    0.02
dtype: float64

In [24]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [25]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

In [26]:
model.fit(X_train, df_train.above_average)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [27]:
y_pred = model.predict_proba(X_val)[:, 1]

In [28]:
y_pred = (y_pred >= 0.5)

In [29]:
round(accuracy_score(df_val.above_average, y_pred),2)

0.95

In [30]:
dv.get_feature_names_out()

array(['city_mpg', 'engine_cylinders', 'engine_hp', 'highway_mpg',
       'make=acura', 'make=alfa_romeo', 'make=aston_martin', 'make=audi',
       'make=bentley', 'make=bmw', 'make=bugatti', 'make=buick',
       'make=cadillac', 'make=chevrolet', 'make=chrysler', 'make=dodge',
       'make=ferrari', 'make=fiat', 'make=ford', 'make=genesis',
       'make=gmc', 'make=honda', 'make=hummer', 'make=hyundai',
       'make=infiniti', 'make=kia', 'make=lamborghini', 'make=land_rover',
       'make=lexus', 'make=lincoln', 'make=lotus', 'make=maserati',
       'make=maybach', 'make=mazda', 'make=mclaren', 'make=mercedes-benz',
       'make=mitsubishi', 'make=nissan', 'make=oldsmobile',
       'make=plymouth', 'make=pontiac', 'make=porsche',
       'make=rolls-royce', 'make=saab', 'make=scion', 'make=spyker',
       'make=subaru', 'make=suzuki', 'make=tesla', 'make=toyota',
       'make=volkswagen', 'make=volvo', 'model=100', 'model=124_spider',
       'model=190-class', 'model=1_series', 'model

In [31]:
categorical_columns + numerical

['make',
 'model',
 'transmission_type',
 'vehicle_style',
 'engine_hp',
 'year',
 'engine_cylinders',
 'highway_mpg',
 'city_mpg']

In [32]:
def new_model(columnas):
    train_dict = df_train[columnas].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    val_dict = df_val[columnas].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    model.fit(X_train, df_train.above_average)
    y_pred = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred >= 0.5)
    return round(accuracy_score(df_val.above_average, y_pred),2)

In [33]:
remo=['year','engine_hp','transmission_type','city_mpg']
print('con_variables',new_model(remo)) 

con_variables 0.89


In [34]:
for c in remo :
    new_col=remo.copy()
    new_col.remove(c)
    print(c,new_model(new_col)) 

year 0.89
engine_hp 0.74
transmission_type 0.88
city_mpg 0.88


In [35]:
price_logs_train = np.log1p(df_train.price)
price_logs_val = np.log1p(df_val.price)

In [36]:
dv = DictVectorizer(sparse=True)
train_dict = df_train[categorical_columns + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
val_dict = df_val[categorical_columns + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [37]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [38]:
for c in [0, 0.01, 0.1, 1, 10]:
    model=Ridge(alpha=c,solver='sag', random_state=42)
    model.fit(X_train, price_logs_train)
    y_pred = model.predict(X_val)
    print( c, round(rmse(price_logs_val, y_pred),2))

0 0.25
0.01 0.25
0.1 0.26
1 0.26
10 0.34
